In [5]:
import findspark
findspark.init()
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
import pandas as pd
from pyspark.sql.window import Window
import pyspark.sql.functions as sf
from pyspark.sql.functions import concat_ws
from datetime import datetime, timedelta
import os

In [7]:
spark = SparkSession.builder.config("spark.driver.memory", "8g").config("spark.executor.cores", 8).getOrCreate()

def category_AppName(df):
    df = df.withColumn("Type", when(col("AppName")=="CHANNEL","Truyen hinh")
                       .when(col("AppName")=="RELAX", "Giai Tri")
                       .when(col("AppName")=="CHILD", "Thieu Nhi")
                       .when((col("AppName")=="FIMS")|(col("AppName")=="VOD"), "Phim Truyen")
                       .when((col("AppName")=="KPLUS")|(col("AppName")=="SPORT"), "The Thao")
                      )
    df = df.select('Contract', 'Type', 'TotalDuration')
    df = df.filter(df.Type != 'Error')
    df = df.filter(df.Contract != '0')
    return df

def most_watch(df):
    df=df.withColumn("MostWacth",greatest(col("Giai Tri"),col("Phim Truyen"),col("The Thao"),col("Thieu Nhi"),col("Truyen Hinh")))
    df=df.withColumn("MostWacth",
                    when(col("MostWacth")==col("Truyen Hinh"),"Truyen Hinh")
                    .when(col("MostWacth")==col("Phim Truyen"),"Phim Truyen")
                    .when(col("MostWacth")==col("The Thao"),"The Thao")
                    .when(col("MostWacth")==col("Thieu Nhi"),"Thieu Nhi")
                    .when(col("MostWacth")==col("Giai Tri"),"Giai Tri"))
    return df

def customer_taste(df):
    df = df.withColumn("Taste", concat_ws("-",
                                          when(col("Giai Tri").isNotNull(), lit("Giai Tri")),
                                          when(col("Phim truyen").isNotNull(), lit("Phim truyen")),
                                          when(col("The Thao").isNotNull(), lit("The Thao")),
                                          when(col("Thieu Nhi").isNotNull(), lit("Thieu Nhi")),
                                          when(col("Truyen Hinh").isNotNull(), lit("Truyen Hinh"))))
    return df

def convert_to_datevalue(string):
    date_value = datetime.strptime(string, "%Y%m%d").date()
    return date_value

def convert_to_stringvalue(date):
    string_value = date.strftime("%Y%m%d")
    return string_value

def date_range(start_date, end_date):
    date_list = []
    current_date = start_date
    while(current_date <= end_date):
        date_list.append(convert_to_stringvalue(current_date))
        current_date += timedelta(days=1)
    return date_list

def generate_range_date(start_date, end_date):
    start_date = convert_to_datevalue(start_date)
    end_date = convert_to_datevalue(end_date)
    date_list = date_range(start_date, end_date)
    return date_list

def ETL_1_DAY(path, path_day):
    print('------------------------')
    print('Read data from Json file')
    print('------------------------')
    df = spark.read.json(path+path_day+".json")
    print('------------------------')
    print('Category AppName')
    print('------------------------')
    df = df.select("_source.*")
    df = category_AppName(df)
    print('------------------------')
    print('Pivoting data')
    print('------------------------')
    df = df.groupBy("Contract").pivot("Type").sum("TotalDuration")
    print('------------------------')
    print('Find most watch')
    print('------------------------')
    df = most_watch(df)
    print('------------------------')
    print('Find customer taste')
    print('------------------------')
    df = customer_taste(df)
    df = df.withColumn("Date", to_date(lit(path_day), "yyyyMMdd"))
    print('------------------------')
    return df

def import_to_postgresql(result):
    url = 'jdbc:postgresql://' + 'localhost' + ':' + '5432' + '/' + 'test_etl'
    driver = "org.postgresql.Driver"
    user = 'postgres'
    password = ''
    result.write.format('jdbc').option('url', url).option('driver', driver).option('dbtable', 'customer_statistic').option('user', user).option('password', password).mode('append').save()
    return print("Data import successfully")

def main(path, save_path):
    dir_list = os.listdir(path)
    start_date = input("Nhap ngay bat dau: ")
    end_date = input("Nhap ngay ket thuc: ")
    date_list = generate_range_date(start_date, end_date)
    # print("ETL data file: " + date_list[0]+".json")
    result = ETL_1_DAY(path, date_list[0])

    for x in date_list:
        for y in dir_list:
            if y == dir_list[0]:
                continue
            if x in y:
                print("ETL data file: " +y)
                result = result.union(ETL_1_DAY(path, x))
    print('-----------------------------')
    print('Showing data')
    print('-----------------------------') 
    result.show(10)
    print('-----------------------------')
    print('Saving csv output')
    print('-----------------------------')
    #result.repartition(1).write.csv(save_path, mode='overwrite', header=True)
    print('-----------------------------')
    print('Import result to mysql')
    print('-----------------------------')
    import_to_postgresql(result)
    print("Finished job")
    return result
    

path = "D:/study_de/dataset/log_content/"
save_path = "D:/study_de/dataset/output_2"
df = main(path,save_path)

Nhap ngay bat dau:  20220401
Nhap ngay ket thuc:  20220430


------------------------
Read data from Json file
------------------------
------------------------
Category AppName
------------------------
------------------------
Pivoting data
------------------------
------------------------
Find most watch
------------------------
------------------------
Find customer taste
------------------------
------------------------
ETL data file: 20220402.json
------------------------
Read data from Json file
------------------------
------------------------
Category AppName
------------------------
------------------------
Pivoting data
------------------------
------------------------
Find most watch
------------------------
------------------------
Find customer taste
------------------------
------------------------
ETL data file: 20220403.json
------------------------
Read data from Json file
------------------------
------------------------
Category AppName
------------------------
------------------------
Pivoting data
------------------------
--

NameError: name 'import_to_mysql' is not defined